In [1]:
!pip install openai


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [2]:
from dotenv import load_dotenv
import os

In [3]:
load_dotenv()  # Load environment variables from the .env file

True

In [4]:
!pip install python-dotenv


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [5]:
api_key = os.getenv('OPENAI_API_KEY')

In [6]:
!pip install OpenAI


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip


# Expression parsing

In [107]:
def extract_structure(latex_expression):
    
    client = OpenAI()
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "system",
                "content": "You are a helpful assistant that parses LaTeX expressions and categorizes their components "
                    "into the following structure: Variables/Constants, Functions, Operators, Brackets, and Misc. "
                    "For each category, provide a list of items found in the LaTeX expression. "
                    "Ensure that any mathematical functions, such as 'max', 'min', 'sin', 'cos', and custom functions like 'U_u', are correctly identified under the 'Functions' category. "
                    "Variables are any symbols or letters that represent quantities, such as 't', 'x', or 'omega_t^o'. "
                    "Constants include numeric values like '-0.20', '0.22', and '0.0361'. "
                    "Operators include symbols like '>', '>=', '+', '-', '*', and '/' that indicate operations between variables or constants. "
                    "Brackets include any grouping symbols such as '(', ')', '[', ']', '{', and '}'. "
                    "Misc should be used for any elements that do not fit into the other categories. "
                    "The output should be a JSON object with the following keys: 'Variables/Constants', 'Functions', 'Operators', 'Brackets', 'Misc'. "
                    "Numbers also fall under constants and variables, and convert the LaTeX into mathematical expressions without LaTeX-specific formatting."
                    "output should be a json dictionary with categories as keys and lists of items as values"
            },
            {
                "role": "user",
                "content": f"Please parse the following LaTeX expression and provide the categorized output structure: {latex_expression}"
            }
        ],
        max_tokens=500,
        temperature=0.5,
    )

    return response.choices[0].message.content

# Provided LaTeX expression
latex_expression = r"t \in [0.0361, 1.000] \rightarrow &  \
U_u(\omega_t^o) \geq \max\left( u, Q_{U_{\Omega^o_t}} (-0.10 \cdot t + 0.64)\right)"

# Get the output structure from the API
output_structure = extract_structure(latex_expression)
print(output_structure)

{
    "Variables/Constants": ["t", "0.0361", "1.000", "\\omega_t^o", "u", "Q_{U_{\\Omega^o_t}}", "-0.10", "0.64"],
    "Functions": ["U_u", "\\max"],
    "Operators": ["\\in", "\\rightarrow", "\\geq", "\\cdot", "+"],
    "Brackets": ["[", "]", "(", ")", "{"],
    "Misc": ["&"]
}


In [51]:
output_structure

'{\n  "Variables/Constants": [\n    "t",\n    "0.0361",\n    "1.000",\n    "u",\n    "0.10",\n    "0.64"\n  ],\n  "Functions": [\n    "U_u",\n    "max",\n    "Q_{U_{\\\\Omega^o_t}}"\n  ],\n  "Operators": [\n    "\\\\in",\n    "\\\\rightarrow",\n    "\\\\geq",\n    "\\\\cdot",\n    "+"\n  ],\n  "Brackets": [\n    "[",\n    "]",\n    "(",\n    ")",\n    "\\\\left(",\n    "\\\\right)"\n  ],\n  "Misc": [\n    ","\n  ]\n}'

In [63]:
def convert_output_to_latex(output_structure):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "system",
                "content": "you are given a JSON format which contains "
            },
            {
                "role": "user",
                "content": f"Please parse the following LaTeX expression and provide the categorized output structure: {latex_expression}"
            }
        ],
        max_tokens=500,
        temperature=0.5,
    )

    return response.choices[0].message.content
    



# Extracting the semantic meaning

In [115]:
parsed_output = extract_structure(latex_expression)
prompt_semantics = (
        "The following is a JSON object with categorized components of a LaTeX expression. "
        "Please provide detailed semantic meanings for each item in the JSON. The categories are: "
        "Variables/Constants, Functions, Operators, Brackets, and Misc. "
        "For each item, provide its name and a detailed description of its meaning.\n\n"
        f"{parsed_output}\n\n"
        "Provide the output in the following JSON format:"
        "{"
        '  "Variables/Constants": ['
        '    {"name": "item_name", "meaning": "item_meaning"}'
        '  ],'
        '  "Functions": '
        '    {"name": "item_name", "meaning": "item_meaning"}'
        '  ],'
        '  "Operators": ['
        '    {"name": "item_name", "meaning": "item_meaning"}'
        '  ],'
        '  "Brackets": ['
        '    {"name": "item_name", "meaning": "item_meaning"}'
        '  ],'
        '  "Misc": ['
        '    {"name": "item_name", "meaning": "item_meaning"}'
        '  ]'
        "}"
    )


def extract_semantics(prompt):
    
    
    
    response = client.chat.completions.create(
        model="gpt-4-turbo",
        messages=[
            {
                "role": "system",
                "content": "You are a helpful assistant."
            },
            {
                "role": "user",
                "content": prompt
            }
        ],
        max_tokens=1500,
        temperature=0.5,
    )

    return response.choices[0].message.content

In [55]:
# parsed_output = extract_structure(latex_expression)
# prompt = (
#         "The following is a JSON object with categorized components of a LaTeX expression. "
#         "Please provide detailed semantic meanings for each item in the JSON. The categories are: "
#         "Variables/Constants, Functions, Operators, Brackets, and Misc. "
#         "For each item, provide its name and a detailed description of its meaning.\n\n"
#         f"{parsed_output}\n\n"
#         "Provide the output in the following JSON format:"
#         "{"
#         '  "Variables/Constants": ['
#         '    {"name": "item_name", "meaning": "item_meaning"}'
#         '  ],'
#         '  "Functions": '
#         '    {"name": "item_name", "meaning": "item_meaning"}'
#         '  ],'
#         '  "Operators": ['
#         '    {"name": "item_name", "meaning": "item_meaning"}'
#         '  ],'
#         '  "Brackets": ['
#         '    {"name": "item_name", "meaning": "item_meaning"}'
#         '  ],'
#         '  "Misc": ['
#         '    {"name": "item_name", "meaning": "item_meaning"}'
#         '  ]'
#         "}"
#     )

In [65]:
print(extract_semantics(prompt))

```json
{
  "Variables/Constants": [
    {"name": "t", "meaning": "Typically represents a variable for time in equations."},
    {"name": "0.0361", "meaning": "A constant value, possibly a parameter or a specific measurement."},
    {"name": "1.000", "meaning": "A constant, representing a whole unit, often used in calculations for scaling or baselines."},
    {"name": "u", "meaning": "A variable, possibly representing a specific quantity or parameter in a function."},
    {"name": "0.10", "meaning": "A numerical constant, likely representing a decimal fraction used in calculations."},
    {"name": "0.64", "meaning": "Another numerical constant, possibly representing a probability, fraction, or other parameter in a model or equation."}
  ],
  "Functions": [
    {"name": "U_u", "meaning": "Represents a function named 'U' with 'u' as a parameter. This could be a utility function, or another specific mathematical function depending on the context."},
    {"name": "max", "meaning": "A mathe

In [58]:
output_structure

'{\n  "Variables/Constants": [\n    "t",\n    "0.0361",\n    "1.000",\n    "u",\n    "0.10",\n    "0.64"\n  ],\n  "Functions": [\n    "U_u",\n    "max",\n    "Q_{U_{\\\\Omega^o_t}}"\n  ],\n  "Operators": [\n    "\\\\in",\n    "\\\\rightarrow",\n    "\\\\geq",\n    "\\\\cdot",\n    "+"\n  ],\n  "Brackets": [\n    "[",\n    "]",\n    "(",\n    ")",\n    "\\\\left(",\n    "\\\\right)"\n  ],\n  "Misc": [\n    ","\n  ]\n}'

In [131]:
import spacy

# Load spaCy model (optional if needed for further NLP tasks)
nlp = spacy.load("en_core_web_sm")
semantic_meanings = {
    "t": "Typically represents a variable for time in equations.",
    "u": "A variable, possibly representing a specific quantity or parameter in a function.",
    "U_u": "Represents a function named 'U' with 'u' as a parameter. This could be a utility function, or another specific mathematical function depending on the context.",
    "max": "A mathematical function that returns the maximum value among its arguments.",
    "Q_{U_{\\Omega^o_t}}": "A function 'Q' applied to another function or variable 'U' which itself depends on a set or parameter 'Ω^o_t'. This could represent a complex mathematical or statistical operation, possibly querying or calculating a specific value from a defined set or distribution.",
    "\\in": "A symbol used in set theory to denote membership; 'x \\in A' means x is an element of set A.",
    ">=": "A relational operator meaning 'greater than or equal to'. It is used to denote inequality between two expressions.",
    "\\cdot": "Represents multiplication in mathematical expressions.",
    "+": "The basic arithmetic operator for addition.",
    "[": "Used to denote the start of an interval or a list in mathematical expressions.",
    "]": "Used to denote the end of an interval or a list in mathematical expressions.",
    "(": "Used to start a grouping of expressions, affecting the order of operations in mathematical expressions.",
    ")": "Used to end a grouping of expressions, affecting the order of operations in mathematical expressions.",
    "{": "Used to start a set or a block of expressions, often used in defining functions or sets.",
    "\\rightarrow": "Denotes a function mapping or an implication in logical expressions. In functions, it can describe the mapping from one set to another.",
    "&": "Represents a logical conjunction (AND) in various contexts."
}

def is_number(s):
    """Check if the string represents a number."""
    try:
        float(s)
        return True
    except ValueError:
        return False

def extract_semantic_meanings(parsed_latex):
    # Initialize the result dictionary
    semantic_extraction = {}

    # Iterate over each category and map the names to their meanings
    for category, items in parsed_latex.items():
        semantic_extraction[category] = []
        for item in items:
            if is_number(item):
                meaning = "A numerical constant, likely representing a specific value or parameter in the equation."
            else:
                meaning = semantic_meanings.get(item, f"No meaning defined for {item}")
            semantic_extraction[category].append({"name": item, "meaning": meaning})

    return semantic_extraction

parsed_latex_dict = json.loads(output_structure)
extract_semantic_meanings(parsed_latex_dict)

{'Variables/Constants': [{'name': 't',
   'meaning': 'Typically represents a variable for time in equations.'},
  {'name': '0.0361',
   'meaning': 'A numerical constant, likely representing a specific value or parameter in the equation.'},
  {'name': '1.000',
   'meaning': 'A numerical constant, likely representing a specific value or parameter in the equation.'},
  {'name': '\\omega_t^o', 'meaning': 'No meaning defined for \\omega_t^o'},
  {'name': 'u',
   'meaning': 'A variable, possibly representing a specific quantity or parameter in a function.'},
  {'name': 'Q_{U_{\\Omega^o_t}}',
   'meaning': "A function 'Q' applied to another function or variable 'U' which itself depends on a set or parameter 'Ω^o_t'. This could represent a complex mathematical or statistical operation, possibly querying or calculating a specific value from a defined set or distribution."},
  {'name': '-0.10',
   'meaning': 'A numerical constant, likely representing a specific value or parameter in the equation

In [60]:
prompt_for_rule_based= f"""
You are an AI model trained to translate mathematical and logical expressions into natural language explanations. Given the semantic output of a mathematical expression, your task is to generate a coherent English explanation. 

The semantic output will include various categories such as Functions, Operators, Variables/Constants, and more. Use the provided information to create a detailed, understandable explanation in plain English.

Here is the semantic output:
{semantic_explanations}

Generate a natural language explanation for this semantic output, making sure to clarify each element's role in the expression.

For example, translate the semantic representation: 
t in [0.000, 0.0361) -> U_u(omega_t^o) >= max(Q_U_Omega_t (-0.20 * t + 0.22), u_t_bar)

into the natural language explanation:  
"At time t, within the range [0.000, 0.0361), the function U_u applied to omega_t^o must yield a value that is greater than or equal to the maximum value between Q_U_Omega_t of (-0.20 * t + 0.22) and the threshold utility u_t_bar."

Please follow a similar structure and level of detail for the provided semantic output.
"""

# Creating Rule based explanations

In [75]:
import re

def generate_rule_based_explanation(semantic_extractions):
    # Start with an empty explanation
    explanation = "Ensures that "

    # Create helper functions to handle specific parts of the explanation
    def handle_functions(semantic_extractions):
        func_explanation = ""
        for func in semantic_extractions['Functions']:
            if func['name'] == "U_u":
                func_explanation += "the utility function U applied to "
            elif func['name'] == "max":
                func_explanation += "the maximum of "
        return func_explanation

    def handle_variables_constants(semantic_extractions):
        var_const_explanation = ""
        for var in semantic_extractions['Variables/Constants']:
            if var['meaning'] == "time":
                var_const_explanation += "time t "
            elif var['meaning'] == "velocity":
                var_const_explanation += "velocity variable"
            elif var['meaning'] == "variable":
                var_const_explanation += f"the variable {var['name']} "
            elif var['meaning'] == "domain":
                var_const_explanation += f"the domain {var['name']} "
            elif var['meaning'] == "utility":
                var_const_explanation += f"a utility threshold {var['name']} "
            elif var['meaning'] == "numeric":
                var_const_explanation += f"the constant value {var['name']} "
        return var_const_explanation

    def handle_operators(semantic_extractions):
        op_explanation = ""
        for op in semantic_extractions['Operators']:
            if op['name'] == ">=":
                op_explanation += "is greater than or equal to "
            elif op['name'] == "->":
                op_explanation += "implies that "
            elif op['name'] == "in":
                op_explanation += "is within the range "
            elif op['name'] == "*":
                op_explanation += "multiplied by "
            elif op['name'] == "+":
                op_explanation += "plus "
        return op_explanation

    # Building the explanation by combining parts
    explanation += handle_functions(semantic_extractions)
    explanation += handle_variables_constants(semantic_extractions)
    explanation += handle_operators(semantic_extractions)

    # Add additional context or wrap-up the sentence if necessary
    explanation += " meets either a calculated statistical value or a specified minimum utility requirement."

    # Return the final explanation
    return explanation

# Example usage with the provided semantic extraction
semantic_extractions = {
    'Brackets': [{'meaning': 'bracket', 'name': '['},
                 {'meaning': 'bracket', 'name': ']'},
                 {'meaning': 'bracket', 'name': '('},
                 {'meaning': 'bracket', 'name': ')'}],
    'Functions': [{'meaning': 'function', 'name': 'U_u'},
                  {'meaning': 'function', 'name': 'max'}],
    'Misc': [{'meaning': 'coordinating conjunction, connecting elements of equal rank', 'name': '&'}],
    'Operators': [{'meaning': 'operator', 'name': 'in'},
                  {'meaning': 'operator', 'name': '->'},
                  {'meaning': 'operator', 'name': '>='},
                  {'meaning': 'operator', 'name': '*'},
                  {'meaning': 'operator', 'name': '+'}],
    'Variables/Constants': [{'meaning': 'time', 'name': 't '},
                            {'meaning': 'numeric', 'name': '0.000'},
                            {'meaning': 'numeric', 'name': '0.0361'},
                            {'meaning': 'variable', 'name': 'omega_t^o'},
                            {'meaning': 'domain', 'name': 'Q_U_Omega_t'},
                            {'meaning': 'utility', 'name': 'u_t_bar'},
                            {'meaning': 'numeric', 'name': '-0.20'},
                            {'meaning': 'numeric', 'name': '0.22'}]
}

# Generate the rule-based explanation
explanation = generate_rule_based_explanation(semantic_extractions)
print(explanation)


Ensures that the utility function U applied to the maximum of time t the constant value 0.000 the constant value 0.0361 the variable omega_t^o the domain Q_U_Omega_t a utility threshold u_t_bar the constant value -0.20 the constant value 0.22 is within the range implies that is greater than or equal to multiplied by plus  meets either a calculated statistical value or a specified minimum utility requirement.


In [19]:
#evaluating rule based explanation

In [20]:
#add a function to compare spacy and llm for semantic extraction

In [129]:
import json

def generate_rule_based_explanation_usingGPT(prompt_semantics):
    
    # Extract the semantic meaning using the provided function
    
    
    # Format the semantic extraction into a JSON string
    prompt = f"""
Given the following semantic extraction of a parsed mathematical expression:

{json.dumps(semantic_extraction, indent=2)}

Based on the above semantic extraction, generate a rule-based explanation for the mathematical expression. The explanation should be clear, concise, and similar in structure to the following example:

"Ensures that \( U_u(\omega_t^o) \) must meet either a fixed utility threshold \( u \) or another calculated statistical measure \( Q \) in the subsequent time interval [0.0361, 1.000]."

Please provide the rule-based explanation using the context and relationships identified in the semantic extraction.
"""

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "system",
                "content": "You are an expert in analysis. You generate easy to understand rule-based explanations for mathematical expressions based on their semantic meanings "
            },
            {
                "role": "user",
                "content": prompt
            }
        ],
        max_tokens=500,
        temperature=0.5,
    )
    
    # Return the generated explanation
    return response.choices[0].message.content

    
    
    

In [130]:
semantic_extraction = extract_semantics(prompt_semantics)
print(generate_rule_based_explanation_usingGPT(semantic_extraction))

"Calculates the maximum value between the function \( U_u \) applied to \( \omega_t^o \) and a specific quantile function \( Q \) of a distribution \( U \) indexed by a set or parameter \( \Omega^o_t \), ensuring the result falls within the interval [0.10, 0.64]."


In [77]:
semantic_extractions = extract_semantics(output_structure) 

{'Variables/Constants': [{'name': 't',
   'meaning': 'Typically represents a variable for time in equations.'},
  {'name': '0.0361',
   'meaning': 'A numerical constant, likely representing a specific value or parameter in the equation.'},
  {'name': '1.000',
   'meaning': 'A numerical constant, likely representing a specific value or parameter in the equation.'},
  {'name': '\\omega_t^o', 'meaning': 'No meaning defined for \\omega_t^o'},
  {'name': 'u',
   'meaning': 'A variable, possibly representing a specific quantity or parameter in a function.'},
  {'name': 'Q_{U_{\\Omega^o_t}}',
   'meaning': "A function 'Q' applied to another function or variable 'U' which itself depends on a set or parameter 'Ω^o_t'. This could represent a complex mathematical or statistical operation, possibly querying or calculating a specific value from a defined set or distribution."},
  {'name': '-0.10',
   'meaning': 'A numerical constant, likely representing a specific value or parameter in the equation

In [110]:
import json

# Example string representing the parsed LaTeX dictionary
parsed_latex_str = '{"Variables/Constants": ["t", "0.0361", "1.000", "u", "0.10", "0.64"], "Functions": ["U_u", "max", "Q_{U_{\\\\Omega^o_t}}"], "Operators": ["\\\\in", ">=", "\\\\cdot", "+"], "Brackets": ["[", "]", "(", ")", "{"], "Misc": ["\\\\rightarrow", "&"]}'

# Convert the string to a dictionary
parsed_latex_dict = json.loads(output_structure)

print(parsed_latex_dict)

{'Variables/Constants': ['t', '0.0361', '1.000', '\\omega_t^o', 'u', 'Q_{U_{\\Omega^o_t}}', '-0.10', '0.64'], 'Functions': ['U_u', '\\max'], 'Operators': ['\\in', '\\rightarrow', '\\geq', '\\cdot', '+'], 'Brackets': ['[', ']', '(', ')', '{'], 'Misc': ['&']}


In [117]:
print(extract_semantics(prompt_semantics))

```json
{
  "Variables/Constants": [
    {
      "name": "t",
      "meaning": "Typically represents a variable for time in mathematical expressions."
    },
    {
      "name": "0.0361",
      "meaning": "A constant value, possibly representing a fixed parameter or coefficient in an equation."
    },
    {
      "name": "1.000",
      "meaning": "A constant, often used to represent a whole unit or 100% in equations."
    },
    {
      "name": "u",
      "meaning": "Commonly used as a variable in equations, potentially representing an unknown quantity to be solved for."
    },
    {
      "name": "0.10",
      "meaning": "A numerical constant, possibly representing a decimal fraction in an equation."
    },
    {
      "name": "0.64",
      "meaning": "Another numerical constant, could be used as a probability, fraction, or parameter in a calculation."
    }
  ],
  "Functions": [
    {
      "name": "U_u",
      "meaning": "Represents a function, possibly a utility function or other s